In [108]:
import sys
import pandas as pd
import os
from keplergl import KeplerGl
from opred import isPoiWithinPoly

import pickle
from h3 import h3

In [109]:
df_roads = pd.read_csv("./data/train/roads_dataset.csv")
df_malls = pd.read_csv("./data/train/mall_coord.csv")
df_transport = pd.read_csv("data/train/routes_dataset.csv")
df_population = pd.read_csv("data/train/rosstat_population_all_cities.csv")
df_isochrones = pd.read_csv("data/train/isochrones_walk_dataset.csv")
df_stops = pd.read_csv("data/train/osm_stops.csv")
df_companies = pd.read_csv("data/train/osm_amenity.csv")
df_target = pd.read_csv("data/train/target_hakaton_spb.csv", encoding = "Windows-1251", sep = ";").drop_duplicates()

In [110]:
df_target_n = df_target[['geo_h3_10','target','atm_cnt','atm_category','lat_h3','lon_h3']]

In [111]:
df_target_n = df_target_n.set_index('atm_category')

In [113]:
n = int(input("Введите номер категории(от 1 до 4): "))
nmax = 4
for i in range(1,5):
    if (i % (nmax+1)) != n:
        df_target_n.drop(['category'+str(i)], axis = 0,inplace = True)

Введите номер категории(от 1 до 4): 2


In [114]:
df_target_n = df_target_n.reset_index()

In [115]:
df_target_n = df_target_n.eval('avg= target / atm_cnt')

In [116]:
df_target_avg = df_target_n[['geo_h3_10', 'avg','lat_h3','lon_h3']]

In [117]:
df_figs = df_isochrones[['walk_15min','geo_h3_10']]

In [118]:
df_sum = df_target_avg.merge(df_figs, how = 'left', on = 'geo_h3_10')

In [124]:
df_sum

,geo_h3_10,avg,lat_h3,lon_h3,walk_15min
0,8a10c228820ffff,1162.166667,56.891648,60.605399,"POLYGON ((60.611399 56.900068, 60.609399 56.90..."
1,8a10c2289a0ffff,696.166667,56.911310,60.628781,NaN
2,8a10c228a087fff,933.000000,56.862388,60.591942,"POLYGON ((60.599942 56.866521, 60.598942 56.86..."
3,8a10c228b577fff,2954.500000,56.873786,60.619674,NaN
4,8a10c2293a17fff,2200.000000,56.808738,60.493857,NaN
...,...,...,...,...,...
104,8a1106ab225ffff,493.166667,59.996784,30.272247,"POLYGON ((30.273247 60.006901, 30.266869 60.00..."
105,8a1106ab27b7fff,1608.250000,59.983154,30.265797,NaN
106,8a1106ab2a6ffff,4832.166667,60.000583,30.256464,"POLYGON ((30.262464 60.010729, 30.259464 60.01..."
107,8a1106ab2b07fff,376.233333,59.996672,30.248656,"POLYGON ((30.247656 60.005838, 30.246656 60.00..."


In [119]:
df_malls_t = df_malls.drop(columns = ['Unnamed: 0'])

In [120]:
df_malls_t.rename(columns={'mall_lat':'lat_h3', 'mall_lng':'lon_h3'}, inplace = True)

In [121]:
df_sum_hash = df_sum.merge(df_malls_t, how = 'left', on = ['lat_h3','lon_h3'] )

In [123]:
df_sum_hash

,geo_h3_10,avg,lat_h3,lon_h3,walk_15min
0,8a10c228820ffff,1162.166667,56.891648,60.605399,"POLYGON ((60.611399 56.900068, 60.609399 56.90..."
1,8a10c2289a0ffff,696.166667,56.911310,60.628781,NaN
2,8a10c228a087fff,933.000000,56.862388,60.591942,"POLYGON ((60.599942 56.866521, 60.598942 56.86..."
3,8a10c228b577fff,2954.500000,56.873786,60.619674,NaN
4,8a10c2293a17fff,2200.000000,56.808738,60.493857,NaN
...,...,...,...,...,...
104,8a1106ab225ffff,493.166667,59.996784,30.272247,"POLYGON ((30.273247 60.006901, 30.266869 60.00..."
105,8a1106ab27b7fff,1608.250000,59.983154,30.265797,NaN
106,8a1106ab2a6ffff,4832.166667,60.000583,30.256464,"POLYGON ((30.262464 60.010729, 30.259464 60.01..."
107,8a1106ab2b07fff,376.233333,59.996672,30.248656,"POLYGON ((30.247656 60.005838, 30.246656 60.00..."


In [122]:
df_sum_hash.to_csv("some_data_"+str(n)+".csv")